# Data Cleaning


In [1]:
import pandas as pd
from matplotlib import pyplot as plt 
import numpy as np

First, we load required data from reviews, and remove rows based on several criteria. 

In [2]:
# choosing only the columns that are needed. The column 'Review' contains the text that makes the dataframe heavy. So it is dropped.
reviews = pd.read_csv('../data/reviews.csv', usecols=['AuthorId', 'ReviewId', 'RecipeId', 'Rating'])
# Data analysis shows 'Rating'=0 is not neccesarily bad rating 
# (Refer to https://www.kaggle.com/code/gemmin/sentiment-analysis)
# Removing 0 ratings
reviews = reviews[reviews['Rating']!=0]
# Only reviews by active reviewers (selected_authors) are accepted, as the model will be based on active reviewers.
# Active reviewer is whoever has more than a minimum number of reviews in the data
review_counts = reviews.groupby('AuthorId').count()
selected_authors = review_counts.loc[review_counts['ReviewId']>=10].index.values
reviews = reviews.loc[reviews['AuthorId'].isin(selected_authors)]

print('number of selected reviews:', reviews['ReviewId'].nunique())
print('number of selected reviewers:', reviews['AuthorId'].nunique())
print('number of selected recipes:', reviews['RecipeId'].nunique())
 

number of selected reviews: 978879
number of selected reviewers: 14544
number of selected recipes: 238699


Now the ratings are normalized based on the reviewer ratings statistics


In [3]:
# substracting mean of reviewer's ratings
reviews['Rating'] -= reviews.groupby('AuthorId')['Rating'].transform(np.mean)
# deviding by std of reviewer's ratings
reviews['Rating'] /= reviews.groupby('AuthorId')['Rating'].transform(np.std)
reviews.describe()

,ReviewId,RecipeId,AuthorId,Rating
count,9.788790e+05,978879.000000,9.788790e+05,9.433290e+05
mean,7.080768e+05,153753.910811,1.264780e+07,1.108753e-17
std,4.470185e+05,128094.099809,1.541074e+08,9.931838e-01
min,7.000000e+00,38.000000,1.533000e+03,-3.465545e+01
25%,3.264205e+05,49047.000000,1.029370e+05,1.066013e-01
50%,6.674580e+05,112362.000000,2.275860e+05,3.495521e-01
75%,1.072974e+06,233848.000000,4.650560e+05,5.175935e-01
max,2.090332e+06,541030.000000,2.002872e+09,3.015113e+00


In [4]:
reviews

,ReviewId,RecipeId,AuthorId,Rating
1,7,4384,1634,-0.215939
16,42,5466,2059,-2.788980
21,47,780,2312,0.592961
25,53,51964,2312,0.592961
29,59,7205,2178,-2.244107
...,...,...,...,...
1401921,2090279,53605,2002480537,0.385164
1401922,2090280,73866,843817,0.171517
1401923,2090281,437232,843817,0.171517
1401942,2090301,339905,35251,0.504956


creating train and test datasets

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(reviews, test_size=0.2)
train.to_csv('../data/reviews_train.csv')
test.to_csv('../data/reviews_test.csv')